In [38]:
import json
import instruction_dataset

In [4]:
# load instruction dataset
with open("instruction-data.json", "r") as file:
    data = json.load(file)

print("Number of entries: ", len(data))
print("Example entry:\n", data[50])


Number of entries:  1100
Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


In [5]:
import torch
import tiktoken

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
dataset = instruction_dataset.InstructionDataset(data, tokenizer)
dataset.__getitem__(0)

[[21106,
  318,
  281,
  12064,
  326,
  8477,
  257,
  4876,
  13,
  19430,
  257,
  2882,
  326,
  20431,
  32543,
  262,
  2581,
  13,
  220,
  198,
  198,
  21017,
  46486,
  25,
  198,
  36,
  2100,
  4985,
  262,
  1708,
  9546,
  416,
  25449,
  340,
  656,
  262,
  24993,
  1813,
  13,
  198,
  198,
  21017,
  23412,
  25,
  198,
  19503,
  521,
  14610,
  1545,
  198,
  198,
  21017,
  18261,
  25,
  198,
  464,
  24993,
  286,
  262,
  1813,
  9546,
  366,
  19503,
  521,
  1,
  318,
  11491,
  11,
  262,
  3376,
  24993,
  318,
  366,
  6726,
  1911],
 [21106,
  318,
  281,
  12064,
  326,
  8477,
  257,
  4876,
  13,
  19430,
  257,
  2882,
  326,
  20431,
  32543,
  262,
  2581,
  13,
  220,
  198,
  198,
  21017,
  46486,
  25,
  198,
  18378,
  262,
  1708,
  6827,
  329,
  23491,
  13,
  198,
  198,
  21017,
  23412,
  25,
  198,
  1544,
  467,
  284,
  262,
  3952,
  790,
  1110,
  13,
  198,
  198,
  21017,
  18261,
  25,
  198,
  1544,
  2925,
  284,
  262,
  3952,
 

In [8]:
test_input = dataset.format_input(data[50])
response = f"\n\n### Response:\n{data[50]['output']}"
print(test_input + response)

Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


In [9]:
test_input = dataset.format_input(data[999])
response = f"\n\n### Response:\n{data[999]['output']}"
print(test_input + response)

Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


In [ ]:
# split data into training, test, and validation sets
train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.1)
val_portion = len(data) - train_portion - test_portion

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

print("Training set length: ", len(train_data))
print("Test set length: ", len(test_data))
print("Validation set length: ", len(val_data))

Training set length:  935
Test set length:  110
Validation set length:  55


In [39]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (inputs_1, inputs_2, inputs_3)

inputs, targets = dataset.collate(batch)
print(inputs)
print()
print(targets)

TypeError: cannot unpack non-iterable NoneType object